In [1]:
import psycopg2
from faker import Faker
import random
import duckdb
import pandas as pd
from datetime import datetime, timedelta, time

In [2]:
# Connect to PostgreSQL database
connection = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
connection.autocommit = True
cursor = connection.cursor()

cursor.execute("SELECT * FROM CUSTOMER")
rows = cursor.fetchall()
print(rows)

[]


In [3]:
cursor.execute("SELECT * FROM CUSTOMER")
rows = cursor.fetchall()
print(rows)

[]


In [4]:
# Initialize Faker
fake = Faker()
fake.seed_instance(42)  # Set seed for reproducibility

In [5]:
# Set pre-defined lists of fake data from which we pull data 

# Aircraft lLsts 
aircraft_types = [
    "Boeing 737", "Boeing 747", "Boeing 777", "Airbus A320", 
    "Airbus A380", "Embraer E190", "Bombardier CRJ700", "Cessna 172", 
    "Piper PA-28", "Gulfstream G650"
]

# Aircraft Companies 
aircraft_companies = [
    "American Airlines", "Delta Air Lines", "United Airlines", "Lufthansa", 
    "Air France", "Qatar Airways", "Singapore Airlines", "British Airways", 
    "Emirates", "Cathay Pacific"
]

# Flight status list 
flight_statuses = ["scheduled", "delayed", "canceled"]

# Airports list 
airports = ["JFK", "LAX", "ORD", "LHR", "CDG", "SIN", "DXB"]

# Maintenance type list 
maintenance_types = ["Scheduled", "Unscheduled"]

# Slot types 
slot_types = ['Flight', 'Maintenance']

# Reporter class list 
reporter_classes = ["Pilot", "Maintenance Personnel"]

# Work Order Status list 
work_order_statuses = ["in-progress", "pending", "completed"]

# AOS Types 
aos_types = ["MaintenanceService", "RevisionService"]

# OI Types 
oi_types = ["delayGenerating", "safetyGenerating"]

# Task Types 
task_types = ["Inspection", "Repair", "Servicing"]

# List of realistic delay codes (limited to 10 characters)
delay_codes = [
    "A1",  # Weather-related delays
    "B2",  # Aircraft mechanical issues
    "C3",  # Crew-related delays
    "D4",  # Airport operational issues
    "E5",  # Security delays
    "F6",  # Maintenance issues
    "G7",  # Late arrival of aircraft
    "H8",  # Late check-in of passengers
    "I9",  # Missing luggage
    "J10"  # Air traffic control delays
]

In [6]:
# Generate Customer Data
def generate_customer_data(num_entries=100):
    customers = []
    for _ in range(num_entries):
        # Generate basic customer data
        customer_id = fake.unique.random_int(min=1, max=10000)
        customer_name = fake.name()
        email = fake.unique.email()
        phone_number = fake.random_number(digits=10)
        address = fake.address()
        
        # Generate frequent_flyer and miles fields
        frequent_flyer = random.choice([True, False])
        miles = fake.random_int(min=1000, max=50000) if frequent_flyer else None  # Assign miles only if frequent_flyer is True

        # Append customer data to list as a tuple
        customer = (customer_id, customer_name, email, phone_number, address, frequent_flyer, miles)
        customers.append(customer)

    return customers

customer_data = generate_customer_data(1000) 

In [7]:
def generate_aircraft_data(num_entries=100):
    aircrafts = []
    for _ in range(num_entries):
        aircraft = (
            fake.unique.random_int(min=1, max=10000),  # aircraft_id
            random.choice(aircraft_types),  # aircraft_type
            random.choice(aircraft_companies),  # aircraft_company
            random.randint(50, 300)  # capacity
        )
        aircrafts.append(aircraft)
    return aircrafts

aircraft_data = generate_aircraft_data(200)

In [8]:
from datetime import timedelta

# Adjusted function to generate realistic AircraftSlot data
def generate_aircraft_slot_data(num_entries=50, aircraft_ids=None):
    slots = []
    for _ in range(num_entries):
        slot_type = random.choice(slot_types)  # Choose either 'Flight' or 'Maintenance'
        
        # Set start_time as a random datetime
        # Generate a start time between the beginning of this year and up to one year from today
        start_time = fake.date_time_between(start_date='-1y', end_date='+1y')

        
        # Set end_time based on slot_type
        if slot_type == 'Flight':
            # For flights, set end_time between 1 to 15 hours after start_time
            duration = timedelta(hours=random.randint(1, 15))
        else:
            # For maintenance, set end_time between 2 to 48 hours after start_time for variety
            duration = timedelta(hours=random.randint(2, 48))
        
        end_time = start_time + duration

        slot = (
            fake.unique.random_int(min=1, max=10000),  # slot_id
            random.choice(aircraft_ids),  # aircraft_id
            slot_type,
            start_time,
            end_time
        )
        slots.append(slot)
    return slots

# Generate the slot data with realistic start and end times
aircraft_slot_data = generate_aircraft_slot_data(1000, aircraft_ids=[a[0] for a in aircraft_data])

In [9]:
def generate_flight_data(num_entries=20):
    flights = []
    for _ in range(num_entries):
        flight_status = random.choices(
            ["scheduled", "delayed", "canceled"],
            weights=[80, 15, 5],  # 80% scheduled, 15% delayed, 5% canceled
            k=1
        )[0]

        # Generate a random date within the last year and the next year
        date_of_flight = fake.date_between(start_date='-1y', end_date='+1y')

        # Generate a start time with a cutoff at 23:00
        hour = random.randint(0, 22)  # Ensure the departure time is before 23:00
        minute = random.randint(0, 59)
        actual_departure_time = time(hour, minute)

        # Ensure the flight duration is at least 1 hour and does not exceed 23:59
        max_duration_hours = 23 - hour  # Max hours to keep arrival within the same day
        duration_hours = random.randint(1, min(max_duration_hours, 12))
        duration = timedelta(hours=duration_hours)
        actual_arrival_time = (datetime.combine(date_of_flight, actual_departure_time) + duration).time()

        
        flight = (
            fake.unique.random_int(min=1, max=10000),  # flight_id
            random.choice(airports),  # departure_airport
            random.choice(airports),  # arrival_airport (ensure it’s different from departure)
            random.choice(delay_codes) if flight_status == "delayed" else None,  # delay_code only for delayed flights
            random.randint(0, 250),  # number_of_passengers
            random.randint(0, 10),  # number_of_cabin_crew
            random.randint(0, 5),  # number_of_flight_crew
            flight_status,  # flight_status
            date_of_flight,  # date_of_flight
            actual_departure_time,  # actual_departure_time
            actual_arrival_time  # actual_arrival_time
        )
        flights.append(flight)
    return flights

# Generate flight data
flight_data = generate_flight_data(1200)

In [10]:
def generate_flight_slot_data(flight_ids=None, slot_ids=None):
    flight_slots = []
    for flight_id, slot_id in zip(flight_ids, slot_ids):
        flight_slot = (
            slot_id,  # slot_id from AircraftSlot
            flight_id  # flight_id from Flight
        )
        flight_slots.append(flight_slot)
    return flight_slots

# Filter slot_ids where slot_type is "Flight"
flight_slot_ids = [s[0] for s in aircraft_slot_data if s[2] == 'Flight']  # slot_id is at index 0, slot_type is at index 2

# Generate FlightSlot data with only flight-related slot_ids
flight_slot_data = generate_flight_slot_data(
    flight_ids=[f[0] for f in flight_data],  # use flight_id from Flight data
    slot_ids=flight_slot_ids[:len(flight_data)]  # use only as many slot_ids as needed, filtered by "Flight"
)

In [11]:
def generate_booking_data(num_entries=30, customer_ids=None):
    generated_ids = set()
    bookings = []
    
    for _ in range(num_entries):
        # Retry to generate a unique booking_id within a limited number of attempts
        for _ in range(100):  
            booking_id = fake.random_int(min=1, max=100000)
            if booking_id not in generated_ids:
                generated_ids.add(booking_id)
                booking = (
                    booking_id,
                    random.choice(customer_ids),
                    random.choice(["Economy", "Business", "First"]),
                    round(random.uniform(50, 1000), 2),
                    fake.boolean(),
                    random.choice(["confirmed", "cancelled", "pending"]),
                    fake.boolean(),
                    random.randint(1, 5),
                    fake.date_time_this_year(),
                    fake.date_time_this_year()
                )
                bookings.append(booking)
                break
        else:
            print("Warning: Could not generate unique booking_id within 100 attempts.")
    return bookings

booking_data = generate_booking_data(5000, customer_ids=[c[0] for c in customer_data])

In [12]:
def generate_maintenance_event_data(num_entries=150):
    events = []
    for _ in range(num_entries):
        event = (
            fake.unique.random_int(min=1, max=10000),  # maintenance_event_id
            random.choice(maintenance_types),  # maintenance_type
            random.choice(airports),  # airport
            fake.word()  # subsystem
        )
        events.append(event)
    return events

maintenance_event_data = generate_maintenance_event_data(500)

In [13]:
def generate_maintenance_slot_data(slot_ids, event_ids):
    maintenance_slots = []
    for slot_id in slot_ids:
        entry = (
            slot_id,  # slot_id from AircraftSlot
            random.choice(event_ids),  # maintenance_event_id from MaintenanceEvent
            random.choice([True, False])  # is_scheduled
        )
        maintenance_slots.append(entry)
    return maintenance_slots

In [14]:
def generate_trip_data(booking_data, flight_ids):
    trips = []
    existing_pairs = set()  # Track (booking_id, flight_id) pairs to avoid duplicates

    # Assign each flight to a trip at least once
    for flight_id in flight_ids:
        # Select a booking for the current flight_id
        booking = random.choice(booking_data)
        booking_id = booking[0]
        pair = (booking_id, flight_id)
        
        if pair not in existing_pairs:
            existing_pairs.add(pair)
            trip = (
                fake.unique.random_int(min=1, max=100000),  # Unique trip_id
                booking_id,
                flight_id
            )
            trips.append(trip)

    # Generate additional trips for round-trip bookings
    for booking in booking_data:
        booking_id = booking[0]
        is_roundtrip = booking[6]  # `is_roundtrip` column

        if is_roundtrip:
            for _ in range(100):  # Retry up to 100 times for uniqueness
                flight_id = random.choice(flight_ids)
                pair = (booking_id, flight_id)
                if pair not in existing_pairs:
                    existing_pairs.add(pair)
                    return_trip = (
                        fake.unique.random_int(min=1, max=100000),  # Unique trip_id
                        booking_id,
                        flight_id
                    )
                    trips.append(return_trip)
                    break
            else:
                print(f"Warning: Could not generate a unique return trip for booking_id {booking_id}")

    return trips

trip_data = generate_trip_data(booking_data, [f[0] for f in flight_data])

In [15]:
# Function to generate data for ScheduledMaintenanceEvent
def generate_scheduled_maintenance_event_data(maintenance_event_ids, num_entries=10):
    scheduled_events = []
    for event_id in random.sample(maintenance_event_ids, num_entries):
        aos_type = random.choice(["MaintenanceService", "RevisionService"])
        
        # Set duration based on aos_type
        if aos_type == "RevisionService":
            duration = timedelta(hours=random.randint(25, 48))  # More than 24 hours
        else:
            duration = timedelta(hours=random.randint(1, 23))  # Less than 24 hours
        
        scheduled_event = (
            event_id,  # scheduled_maintenance_event_id
            duration,
            aos_type,
            fake.date_time_this_year(),  # forecasted_date
            random.choice(["Weekly", "Monthly", "Quarterly", "Annually"])  # frequency
        )
        scheduled_events.append(scheduled_event)
    return scheduled_events

# Function to generate data for UnscheduledMaintenanceEvent
def generate_unscheduled_maintenance_event_data(maintenance_event_ids, flight_ids, num_entries=10):
    unscheduled_events = []
    for event_id in random.sample(maintenance_event_ids, num_entries):
        oi_type = random.choice(["delayGenerating", "safetyGenerating"])
        
        # Set duration based on oi_type
        duration = None if oi_type == "safetyGenerating" else timedelta(minutes=random.randint(30, 180))
        
        unscheduled_event = (
            event_id,  # unscheduled_maintenance_event_id
            random.choice(flight_ids),  # flight_id
            duration,
            oi_type,
            random.choice(["Pilot", "Maintenance Personnel"]),  # reporter_class
            fake.random_int(min=1, max=1000),  # reporter_id
            fake.date_time_this_year()  # reporting_date
        )
        unscheduled_events.append(unscheduled_event)
    return unscheduled_events

In [16]:
# Adjusted function to generate multiple WorkOrderAOS entries per scheduled event
def generate_work_order_aos_data(scheduled_event_ids, max_orders_per_event=3):
    aos_orders = []
    for event_id in scheduled_event_ids:
        # Generate a random number of work orders for each maintenance event
        num_orders = random.randint(1, max_orders_per_event)
        for _ in range(num_orders):
            aos_order = (
                fake.unique.random_int(min=1, max=10000),  # aos_work_order_id
                event_id,  # scheduled_maintenance_event_id
                random.choice(["Inspection", "Repair", "Servicing"]),  # task_type
                fake.date_time_this_year(),  # execution_date
                random.randint(1, 10),  # number_of_workers
                random.choice(["in-progress", "pending", "completed"])  # status
            )
            aos_orders.append(aos_order)
    return aos_orders

# Adjusted function to generate multiple WorkOrderOI entries per unscheduled event
def generate_work_order_oi_data(unscheduled_event_ids, max_orders_per_event=3):
    oi_orders = []
    for event_id in unscheduled_event_ids:
        # Generate a random number of work orders for each maintenance event
        num_orders = random.randint(1, max_orders_per_event)
        for _ in range(num_orders):
            oi_order = (
                fake.unique.random_int(min=1, max=10000),  # oi_work_order_id
                event_id,  # unscheduled_maintenance_event_id
                fake.word() + " parts",  # required_parts
                random.randint(30, 180),  # estimated_completion_time in minutes
                random.choice(["in-progress", "pending", "completed"])  # status
            )
            oi_orders.append(oi_order)
    return oi_orders

In [17]:
# Define other tables similarly, such as AircraftSlot, MaintenanceEvent, Booking, etc.
def insert_data(query, data):
    try:
        with connection.cursor() as cursor:
            cursor.executemany(query, data)
            connection.commit()
            print(f"{len(data)} records inserted successfully.")
    except Exception as e:
        print(f"Error: {e}")
        connection.rollback()

In [18]:
# Define the insert queries
customer_query = """
    INSERT INTO Customer (customer_id, customer_name, email, phone_number, address, frequent_flyer, miles)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

aircraft_query = """
    INSERT INTO Aircraft (aircraft_id, aircraft_type, aircraft_company, capacity)
    VALUES (%s, %s, %s, %s)
"""

flight_query = """
    INSERT INTO Flight (flight_id, departure_airport, arrival_airport, delay_code, 
                        number_of_passengers, number_of_cabin_crew, number_of_flight_crew, 
                        flight_status, date_of_flight, actual_departure_time, actual_arrival_time)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

aircraft_slot_query = """
    INSERT INTO AircraftSlot (slot_id, aircraft_id, slot_type, start_time, end_time)
    VALUES (%s, %s, %s, %s, %s);
"""

maintenance_slot_query = """
    INSERT INTO MaintenanceSlot (slot_id, m)
    VALUES (%s, %s, %s, %s, %s);
"""

flight_slots_query = """
    INSERT INTO FlightSlot (slot_id, flight_id)
    VALUES (%s, %s);
"""

booking_query = """
    INSERT INTO Booking (booking_id, customer_id, seat_class, price, 
                         payment_status, booking_status, is_roundtrip, 
                         number_passengers, created_at, updated_at)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

trip_query = """
    INSERT INTO Trip (trip_id, booking_id, flight_id)
    VALUES (%s, %s, %s)
"""

maintenance_event_query = """
    INSERT INTO MaintenanceEvent (maintenance_event_id, maintenance_type, airport, subsystem)
    VALUES (%s, %s, %s, %s)
"""

maintenance_slots_query = """
    INSERT INTO MaintenanceSlot (slot_id, maintenance_event_id, is_scheduled)
    VALUES (%s, %s, %s)
"""

scheduled_maintenance_event_query = """
    INSERT INTO ScheduledMaintenanceEvent (scheduled_maintenance_event_id, duration, aos_type, 
                                           forecasted_date, frequency)
    VALUES (%s, %s, %s, %s, %s)
"""

unscheduled_maintenance_event_query = """
    INSERT INTO UnscheduledMaintenanceEvent (unscheduled_maintenance_event_id, flight_id, duration, 
                                             oi_type, reporter_class, reporter_id, reporting_date)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

work_order_aos_query = """
    INSERT INTO WorkOrderAOS (aos_work_order_id, scheduled_maintenance_event_id, task_type, 
                              execution_date, number_of_workers, status)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

work_order_oi_query = """
    INSERT INTO WorkOrderOI (oi_work_order_id, unscheduled_maintenance_event_id, required_parts, 
                             estimated_completion_time, status)
    VALUES (%s, %s, %s, %s, %s)
"""


# Generate and insert data into tables
insert_data(customer_query, customer_data)
insert_data(aircraft_query, aircraft_data)
insert_data(aircraft_slot_query, aircraft_slot_data)
insert_data(flight_query, flight_data)
insert_data(flight_slots_query, flight_slot_data)
insert_data(booking_query, booking_data)
insert_data(maintenance_event_query, maintenance_event_data)

# Retrieve necessary IDs from MaintenanceEvent and Flight tables
cursor.execute("SELECT maintenance_event_id FROM MaintenanceEvent")
maintenance_event_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT flight_id FROM Flight")
flight_ids = [row[0] for row in cursor.fetchall()]

# Generate data for MaintenanceSlot table
cursor.execute("SELECT slot_id FROM AircraftSlot WHERE slot_type = 'Maintenance'")
maintenance_slot_ids = [row[0] for row in cursor.fetchall()]

# Generate maintenance_slot_data 
maintenance_slot_data = generate_maintenance_slot_data(maintenance_slot_ids, maintenance_event_ids)

# Insert maintenance slot data 
insert_data(maintenance_slots_query, maintenance_slot_data)

# Generate data
scheduled_maintenance_data = generate_scheduled_maintenance_event_data(maintenance_event_ids, num_entries=75)
unscheduled_maintenance_data = generate_unscheduled_maintenance_event_data(maintenance_event_ids, flight_ids, num_entries=75)

# Continue inserting 
insert_data(trip_query, trip_data)
insert_data(scheduled_maintenance_event_query, scheduled_maintenance_data )
insert_data(unscheduled_maintenance_event_query, unscheduled_maintenance_data)


# Retrieve necessary IDs from ScheduledMaintenanceEvent and UnscheduledMaintenanceEvent tables
cursor.execute("SELECT scheduled_maintenance_event_id FROM ScheduledMaintenanceEvent")
scheduled_event_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT unscheduled_maintenance_event_id FROM UnscheduledMaintenanceEvent")
unscheduled_event_ids = [row[0] for row in cursor.fetchall()]

# Generate data
work_order_aos_data = generate_work_order_aos_data(scheduled_event_ids, max_orders_per_event=3)
work_order_oi_data = generate_work_order_oi_data(unscheduled_event_ids, max_orders_per_event=3)


insert_data(work_order_aos_query, work_order_aos_data )
insert_data(work_order_oi_query, work_order_oi_data)

1000 records inserted successfully.
200 records inserted successfully.
1000 records inserted successfully.
1200 records inserted successfully.
498 records inserted successfully.
5000 records inserted successfully.
500 records inserted successfully.
502 records inserted successfully.
3670 records inserted successfully.
75 records inserted successfully.
75 records inserted successfully.
161 records inserted successfully.
150 records inserted successfully.


In [19]:
import json

# Function to generate data for CustomerPreference
def generate_customer_preference_data(customer_ids, num_entries=10):
    preferences_data = []
    for _ in range(num_entries):
        preference = {
            "meal": random.choice(["vegetarian", "non-vegetarian", "vegan", "kosher"]),
            "seating": {
                "aisle": random.choice([True, False]),
                "extra_legroom": random.choice([True, False]),
                "seat_near_exit": random.choice([True, False])
            },
            "notifications": {
                "email": random.choice([True, False]),
                "sms": random.choice([True, False])
            }
        }
        
        entry = (
            fake.unique.random_int(min=1, max=10000),  # customer_preference_id
            random.choice(customer_ids),  # customer_id
            json.dumps(preference)  # preferences JSONB
        )
        preferences_data.append(entry)
    return preferences_data

# Function to generate data for AircraftMaintenanceLogs
def generate_aircraft_maintenance_logs(maintenance_event_ids, num_entries=10):
    logs_data = []
    for _ in range(num_entries):
        log = {
            "date": str(fake.date_between(start_date='-2y', end_date='today')),
            "check_type": random.choice(["Full Inspection", "Routine Check", "Safety Audit"]),
            "components_checked": [
                {
                    "name": random.choice(["Engine", "Wings", "Hydraulics"]),
                    "status": random.choice(["Operational", "Requires Service"]),
                    "last_replaced": str(fake.date_between(start_date='-2y', end_date='today'))
                } for _ in range(random.randint(1, 2))
            ]
        }
        
        entry = (
            fake.unique.random_int(min=1, max=10000),  # maintenance_event_log_id
            random.choice(maintenance_event_ids),  # maintenance_event_id
            json.dumps(log)  # maintenance_log JSONB
        )
        logs_data.append(entry)
    return logs_data

# Function to generate data for CustomerFeedback
def generate_customer_feedback_data(customer_ids, num_entries=10):
    feedback_data = []
    for _ in range(num_entries):
        survey = {
            "survey_date": str(fake.date_between(start_date='-2y', end_date='today')),
            "rating": random.randint(1, 5),
            "comments": fake.sentence(),
            "topics": {
                "comfort": random.randint(1, 5),
                "service": random.randint(1, 5),
                "cleanliness": random.randint(1, 5),
                "entertainment": random.randint(1, 5)
            }
        }
        
        entry = (
            fake.unique.random_int(min=1, max=10000),  # feedback_id
            random.choice(customer_ids),  # customer_id
            json.dumps(survey)  # survey JSONB
        )
        feedback_data.append(entry)
    return feedback_data

In [20]:
# Example usage with sample customer_ids and maintenance_event_ids
cursor.execute("SELECT customer_id FROM Customer")
customer_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT maintenance_event_id FROM MaintenanceEvent")
maintenance_event_ids = [row[0] for row in cursor.fetchall()]

customer_preference_data = generate_customer_preference_data(customer_ids, 1000)
aircraft_maintenance_logs_data = generate_aircraft_maintenance_logs(maintenance_event_ids, 1000)
customer_feedback_data = generate_customer_feedback_data(customer_ids, 1000)

# SQL Insert Queries
insert_customer_preference = """
    INSERT INTO CustomerPreference (customer_preference_id, customer_id, preferences)
    VALUES (%s, %s, %s)
"""

insert_aircraft_maintenance_logs = """
    INSERT INTO AircraftMaintenanceLogs (maintenance_event_log_id, maintenance_event_id, maintenance_log)
    VALUES (%s, %s, %s)
"""

insert_customer_feedback = """
    INSERT INTO CustomerFeedback (feedback_id, customer_id, survey)
    VALUES (%s, %s, %s)
"""

try:
    cursor.executemany(insert_customer_preference, customer_preference_data)
    cursor.executemany(insert_aircraft_maintenance_logs, aircraft_maintenance_logs_data)
    cursor.executemany(insert_customer_feedback, customer_feedback_data)
    connection.commit()
    print("Data inserted successfully into CustomerPreference, AircraftMaintenanceLogs, and CustomerFeedback tables.")
except Exception as e:
    print("Error inserting data:", e)
finally: 
    cursor.close()
    # Close the connection
    connection.close()

Data inserted successfully into CustomerPreference, AircraftMaintenanceLogs, and CustomerFeedback tables.


In [21]:
# END 